In [28]:
# https://ldzhangyx.github.io/2020/01/13/music-practice/

In [29]:
# Lakh Pianoroll Dataset

In [1]:
from IPython.display import clear_output
from ipywidgets import interact, IntSlider

import os
import os.path
import random
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import torch
import pypianoroll
from pypianoroll import Multitrack, Track, StandardTrack
from tqdm import tqdm
from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot
import random

C:\Users\ASUS\fluid\bin\libfluidsynth-3.dll


In [2]:
dataset_root = Path("D:/midi_data/lpd_5/lpd_5_cleansed/")
id_list = []
"""for path in os.listdir("D:/midi_data/amg"):
    filepath = os.path.join("D:/midi_data/amg", path)
    if os.path.isfile(filepath):
        with open(filepath) as f:
            id_list.extend([line.rstrip() for line in f])
"""
filepath = os.path.join("D:/midi_data/amg", "project/Electronic/Electronic_Strong_Negative.txt")
if os.path.isfile(filepath):
    with open(filepath) as f:
        id_list.extend([line.rstrip() for line in f])
id_list = list(set(id_list))

In [3]:
# Data
n_tracks = 5  # number of tracks
n_pitches = 72  # number of pitches
lowest_pitch = 24  # MIDI note number of the lowest pitch
n_samples_per_song = 8  # number of samples to extract from each song in the datset
n_measures = 4  # number of measures per sample
beat_resolution = 4  # temporal resolution of a beat (in timestep)
programs = [0, 0, 25, 33, 48]  # program number for each track
is_drums = [True, False, False, False, False]  # drum indicator for each track
track_names = ['Drums', 'Piano', 'Guitar', 'Bass', 'Strings']  # name of each track
tempo = 100

# Training
batch_size = 16
latent_dim = 128
n_steps = 17000

# Sampling
sample_interval = 100  # interval to run the sampler (in step)
n_samples = 4

In [4]:
measure_resolution = 4 * beat_resolution
tempo_array = np.full((4 * 4 * measure_resolution, 1), tempo)
assert 24 % beat_resolution == 0, (
    "beat_resolution must be a factor of 24 (the beat resolution used in "
    "the source dataset)."
)
assert len(programs) == len(is_drums) and len(programs) == len(track_names), (
    "Lengths of programs, is_drums and track_names must be the same."
)

In [5]:
def msd_id_to_dirs(msd_id):
    """Given an MSD ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return os.path.join(msd_id[2], msd_id[3], msd_id[4], msd_id)

In [6]:
class GeneraterBlock(torch.nn.Module):
    def __init__(self, in_dim, out_dim, kernel, stride):
        super().__init__()
        self.transconv = torch.nn.ConvTranspose3d(in_dim, out_dim, kernel, stride)
        self.batchnorm = torch.nn.BatchNorm3d(out_dim)
    
    def forward(self, x):
        x = self.transconv(x)
        x = self.batchnorm(x)
        return torch.nn.functional.relu(x)

In [7]:
class Generator(torch.nn.Module):
    """A convolutional neural network (CNN) based generator. The generator takes
    as input a latent vector and outputs a fake sample."""
    def __init__(self):
        super().__init__()
        self.transconv0 = GeneraterBlock(latent_dim, 256, (4, 1, 1), (4, 1, 1))
        self.transconv1 = GeneraterBlock(256, 128, (1, 4, 1), (1, 4, 1))
        self.transconv2 = GeneraterBlock(128, 64, (1, 1, 4), (1, 1, 4))
        self.transconv3 = GeneraterBlock(64, 32, (1, 1, 3), (1, 1, 1))
        self.transconv4 = torch.nn.ModuleList([
            GeneraterBlock(32, 16, (1, 4, 1), (1, 4, 1))
            for _ in range(n_tracks)
        ])
        self.transconv5 = torch.nn.ModuleList([
            GeneraterBlock(16, 1, (1, 1, 12), (1, 1, 12))
            for _ in range(n_tracks)
        ])

    def forward(self, x):
        x = x.view(-1, latent_dim, 1, 1, 1)
        x = self.transconv0(x)
        x = self.transconv1(x)
        x = self.transconv2(x)
        x = self.transconv3(x)
        x = [transconv(x) for transconv in self.transconv4]
        x = torch.cat([transconv(x_) for x_, transconv in zip(x, self.transconv5)], 1)
        x = x.view(-1, n_tracks, n_measures * measure_resolution, n_pitches)
        return x

In [8]:
class LayerNorm(torch.nn.Module):
    """An implementation of Layer normalization that does not require size
    information. Copied from https://github.com/pytorch/pytorch/issues/1959."""
    def __init__(self, n_features, eps=1e-5, affine=True):
        super().__init__()
        self.n_features = n_features
        self.affine = affine
        self.eps = eps
        if self.affine:
            self.gamma = torch.nn.Parameter(torch.Tensor(n_features).uniform_())
            self.beta = torch.nn.Parameter(torch.zeros(n_features))

    def forward(self, x):
        shape = [-1] + [1] * (x.dim() - 1)
        mean = x.view(x.size(0), -1).mean(1).view(*shape)
        std = x.view(x.size(0), -1).std(1).view(*shape)
        y = (x - mean) / (std + self.eps)
        if self.affine:
            shape = [1, -1] + [1] * (x.dim() - 2)
            y = self.gamma.view(*shape) * y + self.beta.view(*shape)
        return y

In [9]:
class DiscriminatorBlock(torch.nn.Module):
    def __init__(self, in_dim, out_dim, kernel, stride):
        super().__init__()
        self.transconv = torch.nn.Conv3d(in_dim, out_dim, kernel, stride)
        self.layernorm = LayerNorm(out_dim)
    
    def forward(self, x):
        x = self.transconv(x)
        x = self.layernorm(x)
        return torch.nn.functional.leaky_relu(x)

In [10]:
class Discriminator(torch.nn.Module):
    """A convolutional neural network (CNN) based discriminator. The
    discriminator takes as input either a real sample (in the training data) or
    a fake sample (generated by the generator) and outputs a scalar indicating
    its authentity.
    """
    def __init__(self):
        super().__init__()
        self.conv0 = torch.nn.ModuleList([
            DiscriminatorBlock(1, 16, (1, 1, 12), (1, 1, 12)) for _ in range(n_tracks)
        ])
        self.conv1 = torch.nn.ModuleList([
            DiscriminatorBlock(16, 16, (1, 4, 1), (1, 4, 1)) for _ in range(n_tracks)
        ])
        self.conv2 = DiscriminatorBlock(16 * n_tracks, 64, (1, 1, 3), (1, 1, 1))
        self.conv3 = DiscriminatorBlock(64, 64, (1, 1, 4), (1, 1, 4))
        self.conv4 = DiscriminatorBlock(64, 128, (1, 4, 1), (1, 4, 1))
        self.conv5 = DiscriminatorBlock(128, 128, (2, 1, 1), (1, 1, 1))
        self.conv6 = DiscriminatorBlock(128, 256, (3, 1, 1), (3, 1, 1))
        self.dense = torch.nn.Linear(256, 1)

    def forward(self, x):
        x = x.view(-1, n_tracks, n_measures, measure_resolution, n_pitches)
        x = [conv(x[:, [i]]) for i, conv in enumerate(self.conv0)]
        x = torch.cat([conv(x_) for x_, conv in zip(x, self.conv1)], 1)
        x = self.conv2(x)
        x = self.conv3(x)          
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = x.view(-1, 256)
        x = self.dense(x)
        return x

In [11]:
def compute_gradient_penalty(discriminator, real_samples, fake_samples):
    """Compute the gradient penalty for regularization. Intuitively, the
    gradient penalty help stablize the magnitude of the gradients that the
    discriminator provides to the generator, and thus help stablize the training
    of the generator."""
    # Get random interpolations between real and fake samples
    alpha = torch.rand(real_samples.size(0), 1, 1, 1).cuda()
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples))
    interpolates = interpolates.requires_grad_(True)
    # Get the discriminator output for the interpolations
    d_interpolates = discriminator(interpolates)
    # Get gradients w.r.t. the interpolations
    fake = torch.ones(real_samples.size(0), 1).cuda()
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    # Compute gradient penalty
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty 

In [12]:
def train_one_step(d_optimizer, g_optimizer, real_samples):
    """Train the networks for one step."""
    
    latent = torch.randn(batch_size, latent_dim)

    # Transfer data to GPU
    if torch.cuda.is_available():
        real_samples = real_samples.cuda()
        latent = latent.cuda()
    
    # === Train the discriminator ===
    # Reset cached gradients to zero
    d_optimizer.zero_grad()
    # Get discriminator outputs for the real samples
    prediction_real = discriminator(real_samples)
    # Compute the loss function
    # d_loss_real = torch.mean(torch.nn.functional.relu(1. - prediction_real))
    d_loss_real = -torch.mean(prediction_real)
    # Backpropagate the gradients
    d_loss_real.backward()
    
    # Generate fake samples with the generator
    fake_samples = generator(latent)
    # Get discriminator outputs for the fake samples
    prediction_fake_d = discriminator(fake_samples.detach())
    # Compute the loss function
    # d_loss_fake = torch.mean(torch.nn.functional.relu(1. + prediction_fake_d))
    d_loss_fake = torch.mean(prediction_fake_d)
    # Backpropagate the gradients
    d_loss_fake.backward()

    # Compute gradient penalty
    gradient_penalty = 10.0 * compute_gradient_penalty(
        discriminator, real_samples.data, fake_samples.data)
    # Backpropagate the gradients
    gradient_penalty.backward()

    # Update the weights
    d_optimizer.step()
    
    # === Train the generator ===
    # Reset cached gradients to zero
    g_optimizer.zero_grad()
    # Get discriminator outputs for the fake samples
    prediction_fake_g = discriminator(fake_samples)
    # Compute the loss function
    g_loss = -torch.mean(prediction_fake_g)
    # Backpropagate the gradients
    g_loss.backward()
    # Update the weights
    g_optimizer.step()

    return d_loss_real + d_loss_fake, g_loss

In [13]:
data = []
# Iterate over all the songs in the ID list
for msd_id in tqdm(id_list):
    # Load the multitrack as a pypianoroll.Multitrack instance
    song_dir = dataset_root / msd_id_to_dirs(msd_id)
    multitrack = pypianoroll.load(song_dir / os.listdir(song_dir)[0])
    #multitrack = pypianoroll.read(song_dir / os.listdir(song_dir)[0])
    # Binarize the pianorolls
    multitrack.binarize()
    # Downsample the pianorolls (shape: n_timesteps x n_pitches)
    multitrack.set_resolution(beat_resolution)
    # Stack the pianoroll (shape: n_tracks x n_timesteps x n_pitches)
    pianoroll = (multitrack.stack() > 0)
    # Get the target pitch range only
    pianoroll = pianoroll[:, :, lowest_pitch:lowest_pitch + n_pitches]
    # Calculate the total measures
    n_total_measures = multitrack.get_max_length() // measure_resolution
    candidate = n_total_measures - n_measures
    target_n_samples = min(n_total_measures // n_measures, n_samples_per_song)
    # Randomly select a number of phrases from the multitrack pianoroll
    for idx in np.random.choice(candidate, target_n_samples, False):
        start = idx * measure_resolution
        end = (idx + n_measures) * measure_resolution
        # Skip the samples where some track(s) has too few notes
        if (pianoroll.sum(axis=(1, 2)) < 10).any():
            continue
        data.append(pianoroll[:, start:end])
# Stack all the collected pianoroll segments into one big array
random.shuffle(data)
data = np.stack(data)
print(f"Successfully collect {len(data)} samples from {len(id_list)} songs")
print(f"Data shape : {data.shape}")

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  9.87it/s]

Successfully collect 16 samples from 13 songs
Data shape : (16, 5, 64, 72)


In [14]:
selected_data = []
max = len(data)
ind_list = list(range(0, max))
n = 24
if max < n:
    n = max
selected_ind = random.sample(ind_list, n)
selected_ind = sorted(selected_ind)
for ind in selected_ind:
    selected_data.append(data[ind])
selected_data = np.stack(selected_data)

#data = torch.as_tensor(data, dtype=torch.float32)
selected_data = torch.as_tensor(selected_data, dtype=torch.float32)
dataset = torch.utils.data.TensorDataset(selected_data)
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, drop_last=True, shuffle=True)
print(len(selected_data))

16


In [18]:
for i in range(3):
    selected_data = []
    max = len(data)
    ind_list = list(range(0, max))
    n = 24
    if max < n:
        n = max
    selected_ind = random.sample(ind_list, n)
    selected_ind = sorted(selected_ind)
    for ind in selected_ind:
        selected_data.append(data[ind])
    selected_data = np.stack(selected_data)

    #data = torch.as_tensor(data, dtype=torch.float32)
    selected_data = torch.as_tensor(selected_data, dtype=torch.float32)
    dataset = torch.utils.data.TensorDataset(selected_data)
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, drop_last=True, shuffle=True)


    # Create neural networks
    discriminator = Discriminator()
    generator = Generator()

    # Create optimizers
    d_optimizer = torch.optim.Adam(
        discriminator.parameters(), lr=0.001,  betas=(0.5, 0.9))
    g_optimizer = torch.optim.Adam(
        generator.parameters(), lr=0.001, betas=(0.5, 0.9))

    # Prepare the inputs for the sampler, which wil run during the training
    sample_latent = torch.randn(n_samples, latent_dim)

    # Transfer the neural nets and samples to GPU
    if torch.cuda.is_available():
        discriminator = discriminator.cuda()
        generator = generator.cuda()
        sample_latent = sample_latent.cuda()

    # Create an empty dictionary to sotre history samples
    history_samples = {}

    # Create a LiveLoss logger instance for monitoring
    liveloss = PlotLosses(outputs=[MatplotlibPlot(cell_size=(6,2))])

    # Initialize step
    step = 0
    # Create a progress bar instance for monitoring
    n_steps = 17000
    progress_bar = tqdm(total=n_steps, initial=step, ncols=80, mininterval=1)
    # Start iterations
    while step < n_steps + 1:
        # Iterate over the dataset
        for real_samples in data_loader:
            # Train the neural networks
            generator.train()
            d_loss, g_loss = train_one_step(d_optimizer, g_optimizer, real_samples[0])

            # Record smoothened loss values to LiveLoss logger
            if step > 0:
                running_d_loss = 0.05 * d_loss + 0.95 * running_d_loss
                running_g_loss = 0.05 * g_loss + 0.95 * running_g_loss
            else:
                running_d_loss, running_g_loss = 0.0, 0.0
            liveloss.update({'negative_critic_loss': -running_d_loss})
            # liveloss.update({'d_loss': running_d_loss, 'g_loss': running_g_loss})

            # Update losses to progress bar
            progress_bar.set_description_str(
                "(d_loss={: 8.6f}, g_loss={: 8.6f})".format(d_loss, g_loss))

            if step % sample_interval == 0:
                # Get generated samples
                generator.eval()

                samples = generator(sample_latent).cpu().detach().numpy()
                history_samples[step] = samples

                # Display loss curves
                clear_output(True)
                #if step > 0:
                    #liveloss.send()

                # Display generated samples
                samples = samples.transpose(1, 0, 2, 3).reshape(n_tracks, -1, n_pitches)
                tracks = []
                for idx, (program, is_drum, track_name) in enumerate(
                    zip(programs, is_drums, track_names)
                ):
                    pianoroll = np.pad(
                        samples[idx] > 0.5,
                        ((0, 0), (lowest_pitch, 128 - lowest_pitch - n_pitches))
                    )
                    tracks.append(
                        StandardTrack(
                            name=track_name,
                            program=program,
                            is_drum=is_drum,
                            pianoroll=pianoroll
                        )
                    )
                m = Multitrack(
                    tracks=tracks,
                    tempo=tempo_array,
                    resolution=beat_resolution
                )
                

            step += 1
            progress_bar.update(1)
            if step >= n_steps:
                break
    file_name = "Electronic_Strong_Negative"
    pypianoroll.write("midi_file/"+file_name+str(i)+".mid",m)
    # Specify a path to save to
    PATH = "model/"+file_name+str(i)+".pt"
    torch.save({
                'Generator_state_dict': generator.state_dict(),
                'Discriminator_state_dict':discriminator.state_dict(),
                'd_optimizer_state_dict':d_optimizer.state_dict(),
                'g_optimizer_state_dict':g_optimizer.state_dict()
                }, PATH)


(d_loss=-5.482748, g_loss= 0.321404):   1%| | 201/17000 [02:26<1:06:43,  4.20it/
(d_loss=-4.591608, g_loss= 2.413984):   1%| | 202/17000 [02:26<1:06:43,  4.20it/
(d_loss=-5.484363, g_loss= 3.141258):   1%| | 203/17000 [02:26<1:06:43,  4.20it/
(d_loss=-5.484363, g_loss= 3.141258):   1%| | 204/17000 [02:26<1:05:32,  4.27it/
(d_loss=-5.371173, g_loss= 3.643832):   1%| | 204/17000 [02:26<1:05:32,  4.27it/
(d_loss=-6.236157, g_loss= 3.513911):   1%| | 205/17000 [02:26<1:05:31,  4.27it/
(d_loss=-6.601859, g_loss= 0.638586):   1%| | 206/17000 [02:27<1:05:31,  4.27it/
(d_loss=-4.671699, g_loss= 4.321382):   1%| | 207/17000 [02:27<1:05:31,  4.27it/
(d_loss=-6.502062, g_loss= 4.431223):   1%| | 208/17000 [02:27<1:05:31,  4.27it/
(d_loss=-6.502062, g_loss= 4.431223):   1%| | 209/17000 [02:27<1:04:42,  4.33it/
(d_loss=-6.674587, g_loss= 2.458454):   1%| | 209/17000 [02:27<1:04:42,  4.33it/
(d_loss=-4.619405, g_loss= 2.946209):   1%| | 210/17000 [02:28<1:04:41,  4.33it/
(d_loss=-4.790051, g_loss= 

KeyboardInterrupt: 

In [41]:
checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint['Generator_state_dict'])

<All keys matched successfully>

In [51]:
sample_latent = torch.randn(n_samples, latent_dim)
sample_latent = sample_latent.cuda()
samples = generator(sample_latent).cpu().detach().numpy()

In [52]:
samples = samples.transpose(1, 0, 2, 3).reshape(n_tracks, -1, n_pitches)
tracks = []
for idx, (program, is_drum, track_name) in enumerate(
    zip(programs, is_drums, track_names)
):
    pianoroll = np.pad(
        samples[idx] > 0.5,
        ((0, 0), (lowest_pitch, 128 - lowest_pitch - n_pitches))
    )
    tracks.append(
        StandardTrack(
            name=track_name,
            program=program,
            is_drum=is_drum,
            pianoroll=pianoroll
        )
    )
m = Multitrack(
    tracks=tracks,
    tempo=tempo_array,
    resolution=beat_resolution
)

In [53]:
pypianoroll.write("MuseGan6000-9.mid",m) 

In [28]:
"""# Data
n_tracks = 1  # number of tracks
n_pitches = 72  # number of pitches
lowest_pitch = 24  # MIDI note number of the lowest pitch
n_samples_per_song = 8  # number of samples to extract from each song in the datset
n_measures = 4  # number of measures per sample
beat_resolution = 4  # temporal resolution of a beat (in timestep)
programs = [0]  # program number for each track
is_drums = [False]  # drum indicator for each track
track_names = ['Piano']  # name of each track
tempo = 100

# Training
batch_size = 16
latent_dim = 128
n_steps = 9000

# Sampling
sample_interval = 100  # interval to run the sampler (in step)
n_samples = 4"""

"# Data\nn_tracks = 1  # number of tracks\nn_pitches = 72  # number of pitches\nlowest_pitch = 24  # MIDI note number of the lowest pitch\nn_samples_per_song = 8  # number of samples to extract from each song in the datset\nn_measures = 4  # number of measures per sample\nbeat_resolution = 4  # temporal resolution of a beat (in timestep)\nprograms = [0]  # program number for each track\nis_drums = [False]  # drum indicator for each track\ntrack_names = ['Piano']  # name of each track\ntempo = 100\n\n# Training\nbatch_size = 16\nlatent_dim = 128\nn_steps = 9000\n\n# Sampling\nsample_interval = 100  # interval to run the sampler (in step)\nn_samples = 4"